In [2]:
from _Setting import StockSetting
import tushare  as ts
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import json
from pyspark.sql.types import StructType, StructField, StringType, DoubleType
from pyspark.sql.functions import col


In [3]:
settings = StockSetting()
spark = SparkSession.builder \
    .appName("_Test") \
    .master("local[*]") \
    .config('spark.submit.pyFiles', '/work/dev/stock/_Setting.py') \
    .getOrCreate() 

2022-04-13 15:57:17,398 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
data_path = settings.data_all_path+"daily_data_ALL"
daily_data_ALL = spark.read.format("json").load(data_path)

In [5]:
data_path = settings.data_all_path+"daily_basic_data_ALL"
daily_basic_data_ALL = spark.read.format("json").load(data_path)

In [6]:
savepath = settings.data_all_path+"daily_moneyflow_data_ALL"
daily_moneyflow_data_ALL = spark.read.format("json").load(data_path)

2022-04-13 15:58:17,606 WARN datasources.SharedInMemoryCache: Evicting cached table partition metadata from memory due to size constraints (spark.sql.hive.filesourcePartitionFileCacheSize = 262144000 bytes). This may impact query planning performance.


In [7]:
savepath = settings.datasource_path+"stock_basic_main"
stock_basic_main = spark.read.format("json").load(savepath)

In [21]:
# stock_basic_main daily_data_ALL daily_basic_data_ALL daily_moneyflow_data_ALL
stocklist = stock_basic_main.where("list_date < '20190101'")
stock_pe = daily_basic_data_ALL.where("trade_date = '20220304' and pe >0")
print(stock_pe.count())


1879


In [22]:
stocklist = stocklist.alias("list").join(stock_pe.alias("pe"),stocklist.ts_code == stock_pe.ts_code).select( \
    col("pe.*") \
    ,col("list.name") \
    ,col("list.area") \
    ,col("list.industry") \
    ,col("list.market"))

stocklist.show(5)

+---------------+-----+------------+------------+------+--------+-------+-------+-------+---------------+------------+----------+---------+-------------+---------------+------------+--------+----+--------+------+
|        circ_mv|close| float_share|  free_share|    pb|      pe| pe_ttm|     ps| ps_ttm|       total_mv| total_share|trade_date|  ts_code|turnover_rate|turnover_rate_f|volume_ratio|    name|area|industry|market|
+---------------+-----+------------+------------+------+--------+-------+-------+-------+---------------+------------+----------+---------+-------------+---------------+------------+--------+----+--------+------+
|1.08086262133E7|18.26| 591929.1464| 273008.5293|1.3498| 13.8632|13.2045| 4.7734| 4.0206|1.39161060745E7| 762108.7664|  20220304|000776.SZ|       1.3915|         3.0169|        1.39|广发证券|广东|    证券|  主板|
|1.69583999115E7| 4.65|3646967.7229|3031809.0615|1.4349| 35.5016|  7.919| 1.3188| 0.9068|1.78772721141E7|3844574.6482|  20220304|000725.SZ|       0.8194|     

In [23]:
x1 = stocklist.where("circ_mv < 1000000").count() #100
x2 = stocklist.where("circ_mv > 1000000 and circ_mv < 5000000").count()
x3 = stocklist.where("circ_mv > 5000000").count()
print(x1,x2,x3)

909 546 159


In [43]:
daily_basic_data_ALL.select(col("pb"),col("pe"),(col("pb")-col("pe")).alias("AA")).show()

+-------+--------+-------------------+
|     pb|      pe|                 AA|
+-------+--------+-------------------+
| 9.3874|222.3001|-212.91269999999997|
| 9.7743|231.4622|-221.68789999999998|
| 10.263|243.0353|          -232.7723|
|10.6296|251.7151|          -241.0855|
|10.7721|255.0906|          -244.3185|
|10.5685|250.2685|             -239.7|
| 10.263|243.0353|          -232.7723|
|10.5685|250.2685|             -239.7|
|10.1612|240.6242|           -230.463|
| 9.6725|229.0511|          -219.3786|
| 9.5096|225.1934|          -215.6838|
| 9.4282|223.2646|          -213.8364|
| 9.7743|231.4622|-221.68789999999998|
|10.2834|243.5175|          -233.2341|
|10.3852|245.9286|          -235.5434|
|10.3852|245.9286|          -235.5434|
| 10.935|258.9483|-248.01330000000002|
|11.4237|270.5214|-259.09770000000003|
| 11.383| 269.557|-258.17400000000004|
|11.0572|261.8416|-250.78440000000003|
+-------+--------+-------------------+
only showing top 20 rows



In [29]:
# stock_basic_main daily_data_ALL daily_basic_data_ALL daily_moneyflow_data_ALL
daily_data_merge = daily_data_ALL.alias("all") \
    .join(stocklist.alias("stcoklist"),col("all.ts_code") == col("stcoklist.ts_code")) \
    .join(daily_basic_data_ALL.alias("basic"), (col("all.ts_code") == col("basic.ts_code") ) & (col("all.trade_date") == col("basic.trade_date") )) \
    .join(daily_moneyflow_data_ALL.alias("money"),(col("all.ts_code") == col("money.ts_code") ) & (col("all.trade_date") == col("money.trade_date") )) \
    .select(col("all.*") \
    ,col("basic.total_mv"),col("basic.circ_mv"),col("basic.pe"),col("basic.pb"),col("basic.turnover_rate"),col("basic.turnover_rate_f") \
    ,(col("money.buy_sm_amount")-col("money.sell_sm_amount").alias("sm_amount")) \
    ,(col("money.buy_md_amount")-col("money.sell_md_amount").alias("md_amount")) \
    ,(col("money.buy_lg_amount")-col("money.sell_lg_amount").alias("lg_amount")) \
    ,(col("money.buy_elg_amount")-col("money.sell_elg_amount").alias("elg_amount")) \
    )

daily_data_merge.show(5)

+----------+------+-----+-----+-----+-----+-------+---------+----------+---------+--------+-------------+--------------+
|    amount|change|close| high|  low| open|pct_chg|pre_close|trade_date|  ts_code|     vol|turnover_rate|turnover_rate1|
+----------+------+-----+-----+-----+-----+-------+---------+----------+---------+--------+-------------+--------------+
|27237.6637| -0.12|15.38|15.68|15.35|15.68|  -0.77|     15.5|  20010709|000001.SZ|17663.84|       0.1268|        0.1268|
|33897.2571| -0.02|11.09| 11.3|11.01| 11.1|  -0.18|    11.11|  20030630|000001.SZ|30499.07|       0.2164|        0.2164|
|31836.2506|  0.14| 11.3|11.32|11.15|11.18|   1.25|    11.16|  20030708|000001.SZ|28254.23|       0.2005|        0.2005|
|27160.9837|  0.11|  8.0|  8.0|  7.8| 7.88|   1.39|     7.89|  20031110|000001.SZ|34275.28|       0.2432|        0.2432|
|32182.9617|  0.07| 5.79| 5.82| 5.66| 5.73|   1.22|     5.72|  20050720|000001.SZ|55713.35|       0.3953|        0.3953|
+----------+------+-----+-----+-